# MVVM + Hexagonal Architecture in SEVA — Part 1 (Adapters and Usecases)

This notebook explains the **MVVM + Hexagonal** structure used in this codebase. It is written for **university beginners** who have only written basic Python scripts and are now going to work on this project.

**Learning goals**
- Understand the *layers* (Views, ViewModels, UseCases, Adapters, Domain).
- See how **Ports** (interfaces) enable swapping adapters without changing the use case logic.
- Walk through a real workflow: **start experiment → submit via Job adapter**.
- Practice with **mock adapters** so you can experiment safely.

Follow-up: **Part 2** covers Views & ViewModels in detail and shows how they are initialized.


In [ ]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))


## 1) Quick map of the architecture

> Think of this as two ideas combined:
>
> **MVVM** keeps UI state separate from logic.  
> **Hexagonal** (Ports & Adapters) keeps external I/O separate from business logic.

```
UI Layer (Views)         <-- rendering only
    |
ViewModels               <-- UI state + commands
    |
UseCases                 <-- orchestration / workflow
    |
Ports (interfaces)       <-- contract boundary
    |
Adapters                 <-- I/O implementations (HTTP, filesystem, etc.)
    |
External services / devices
```

The SEVA code follows this **strict separation**:
- Views never call network/filesystem APIs.
- ViewModels never build raw HTTP payloads.
- UseCases never perform direct I/O.
- Adapters do all I/O and must match the Port contract.

## 2) Ports and Adapters (the Hexagonal core)

A **Port** is a contract (interface). An **Adapter** is a concrete implementation.

In this repo:
- Ports live in: `seva/domain/ports.py`
- Example port: `JobPort`
- Adapter implementations:
  - `JobRestAdapter` (real HTTP adapter) in `seva/adapters/job_rest.py`
  - `JobRestMock` (offline mock adapter) in `seva/adapters/job_rest_mock.py`

**Important idea:** if two adapters implement the same Port, the use case can swap between them with no changes.

The Port defines at least the **method names and signatures** (e.g., `start_batch`, `poll_group`).
Adapters can translate internally, but they must provide these names so use cases can call them.


## 3) The workflow we will simulate

We will simulate a **start experiment** flow using a mock adapter:

1. Build a typed `ExperimentPlan` in a UseCase (`BuildExperimentPlan`).
2. Submit it through a Port (`JobPort.start_batch`).
3. Read results from the same port using a mock adapter.

This is exactly how the real application works — just without real HTTP.

In [ ]:
# Imports from the repo
from seva.usecases.build_experiment_plan import (
    BuildExperimentPlan,
    ExperimentPlanRequest,
    ModeSnapshot,
    WellSnapshot,
)
from seva.usecases.start_experiment_batch import StartExperimentBatch
from seva.adapters.job_rest_mock import JobRestMock

### Build an ExperimentPlan (UseCase)

The UI never creates raw JSON directly. Instead, it provides **snapshots**, and the
UseCase builds a typed `ExperimentPlan` domain object.

In [ ]:
request = ExperimentPlanRequest(
    experiment_name="Intro Experiment",
    subdir="training",
    client_datetime_override=None,
    wells=["A1"],
    well_snapshots=(
        WellSnapshot(
            well_id="A1",
            modes=(
                ModeSnapshot(
                    name="CV",
                    params={
                        "cv.start_v": -0.2,
                        "cv.vertex1_v": 0.3,
                        "cv.vertex2_v": -0.3,
                        "cv.final_v": 0.0,
                        "cv.scan_rate_v_s": 0.1,
                        "cv.cycles": 2,
                    },
                ),
            ),
        ),
    ),
)

build_plan = BuildExperimentPlan()
plan = build_plan(request)
plan

### Submit via a Port (using a mock adapter)

Now we use **StartExperimentBatch**, which depends only on the `JobPort` interface.
We inject the **mock adapter** so there is no real HTTP.

You can also inject the AMETEK adapter here (see the example below) because it
implements the same `JobPort` method names.

The mock adapter now simulates `progress_pct` during polling so you can see it move from 0 to 100 over about a minute.


In [ ]:
job_port = JobRestMock()  # Adapter implements JobPort
start_batch = StartExperimentBatch(job_port=job_port)
result = start_batch(plan)
result


### Poll status (still through the Port)

The mock adapter stores state in memory, so we can query it just like the real API.

Run the polling cell multiple times (or press "Run" again) to see the progress change.
You can also re-run the submit cell to start a new group and then poll again.


In [ ]:
run_group_id = result.run_group_id
snapshot = job_port.poll_group(run_group_id)
snapshot


## 4) Swapping adapters (same Port, new implementation)

Here is the key: **UseCases do not care** *which* adapter they receive, as long as it
implements the Port contract.

If you swap this:

```python
job_port = JobRestMock()
```

for this:

```python
from seva.adapters.job_rest import JobRestAdapter
from seva.domain.entities import BoxId

job_port = JobRestAdapter(
    base_urls={BoxId("A"): "http://localhost:8000"},
    api_keys={BoxId("A"): ""},
)
```

Then the **same use case** (`StartExperimentBatch`) works with a real backend.

Tip: Use the **Raspberry Pi API-Server IP** instead of http://localhost:8000 to get real example.

### Example: swapping to an AMETEK adapter
Imagine you want to use the AMETEK API instead. 

Although the AMETEK builds experiments differently and returns different
polling shapes, the **JobPort contract** is still the same, so the adapter
**translates** between AMETEK and our domain objects.


In [ ]:
from seva.domain.ports import JobPort, BoxId, RunGroupId
from seva.domain.entities import ExperimentPlan


class AmetekJobAdapter(JobPort):
    """Demo adapter showing that a completely different backend (AMETEK API)
    can be integrated as long as we translate everything inside the adapter.
    """

    # ---------- JobPort ----------

    def start_batch(self, plan: ExperimentPlan):
        print("AMETEK API:")
        print("  - translate ExperimentPlan -> proprietary AMETEK payload")
        print("  - start experiment on Solatron SI 6200")
        print("  - return AMETEK run group id")

        # Totally fake, but contract-compatible
        return "ametek-group-1", {}

    def cancel_run(self, box_id: BoxId, run_id: str):
        print(f"AMETEK API: cancel run {run_id} on box {box_id}")

    def cancel_runs(self, box_to_run_ids):
        print("AMETEK API: cancel multiple runs")
        print(" ", box_to_run_ids)

    def cancel_group(self, run_group_id: RunGroupId):
        print(f"AMETEK API: cancel whole group {run_group_id}")

    def poll_group(self, run_group_id: RunGroupId):
        print(f"AMETEK API: poll status for group {run_group_id}")
        print("AMETEK API: translate proprietary status -> normalized snapshot")

        # Minimal, normalized snapshot
        # If AMETEK API doesn't supprt Progress, return a value of 0% to satisfy the backend contract
        return {
            "boxes": {
                "A": {
                    "runs": [
                        {
                            "run_id": "ametek-run-1",
                            "status": "running",      # Could be aquired by AMETEKAPI.GetStatus(Experiment) from DCPy
                            "started_at": None,
                            "progress_pct": 0.0,      # backend compatibility
                        }
                    ],
                    "phase": "Running",
                    "progress_pct": 0.0,
                }
            },
            "wells": [],
            "progress_pct": 0.0,
        }

    def download_group_zip(self, run_group_id: RunGroupId, target_dir: str):
        print("AMETEK API: download not supported")
        print("AMETEK API: return empty result to keep backend running")
        return target_dir


## 5) Where MVVM fits in this workflow

- **View**: draws UI and sends user actions (no I/O, no domain logic).
- **ViewModel**: collects user inputs, builds request snapshots, calls use cases.
- **UseCase**: builds plans and orchestrates workflows (`BuildExperimentPlan`, `StartExperimentBatch`).
- **Port**: contract for I/O (`JobPort`).
- **Adapter**: real or mock I/O (`JobRestAdapter`, `JobRestMock`).

This is why the **core logic stays testable**: you can run most of the system using mocks.

## 6) What to explore next

Try these exercises:
1. Add a **second well** to the request and see how `JobRestMock` returns more runs.  
2. Check out the Code, understand how `ExperimentPlanRequest` is build and what each domain object defines
3. Create your **own** mock adapter that implements the same `JobPort` methods.

#### (Spoiler Warning) Code Solution for 1.

In [249]:
request = ExperimentPlanRequest(
    experiment_name="Intro Experiment",
    subdir="training",
    client_datetime_override=None,
    wells=["A1","A2"],
    well_snapshots=(
        WellSnapshot(
            well_id="A1",
            modes=(
                ModeSnapshot(
                    name="CV",
                    params={
                        "cv.start_v": -0.2,
                        "cv.vertex1_v": 0.3,
                        "cv.vertex2_v": -0.3,
                        "cv.final_v": 0.0,
                        "cv.scan_rate_v_s": 0.1,
                        "cv.cycles": 2,
                    },
                ),
            ),
        ),
        WellSnapshot(
            well_id="A2",
            modes=(
                ModeSnapshot(
                    name="CV",
                    params={
                        "cv.start_v": -0.1,
                        "cv.vertex1_v": 0.2,
                        "cv.vertex2_v": -0.2,
                        "cv.final_v": 0.0,
                        "cv.scan_rate_v_s": 0.05,
                        "cv.cycles": 2,
                    },
                ),
            ),
        ),
    ),
)

build_plan = BuildExperimentPlan()
plan = build_plan(request)
plan

ExperimentPlan(meta=PlanMeta(experiment='Intro Experiment', subdir='training', client_dt=ClientDateTime(value=datetime.datetime(2026, 2, 5, 13, 42, 33, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'Pacific Normalzeit'))), group_id=GroupId(value='Intro_Experiment_training__20260205_134233__58XH'), make_plot=False, tia_gain=None, sampling_interval=None), wells=[WellPlan(well=WellId(value='A1'), modes=[ModeName(value='CV')], params_by_mode={ModeName(value='CV'): CVParams(flags={}, start=-0.2, vertex1=0.3, vertex2=-0.3, end=0.0, scan_rate=0.1, cycles=2)}), WellPlan(well=WellId(value='A2'), modes=[ModeName(value='CV')], params_by_mode={ModeName(value='CV'): CVParams(flags={}, start=-0.1, vertex1=0.2, vertex2=-0.2, end=0.0, scan_rate=0.05, cycles=2)})])